# Mikrofon Geometrisi Kontrolü

Bu notebook, `micgeom.xml` dosyasındaki mikrofon pozisyonlarını yükler ve görselleştirir.

**Amaç:**
- XML dosyasının doğru parse edildiğini doğrulamak
- Mikrofon dizisi geometrisini görselleştirmek
- Acoular entegrasyonunu test etmek

In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# Proje src klasörünü path'e ekle
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root / 'src'))

from geometry.parser import MicGeometryParser

print("✓ Import başarılı")
print(f"Proje dizini: {project_root}")

## 1. XML Dosyasını Yükle

In [ ]:
# XML dosya yolu
xml_path = project_root / "config" / "micgeom.xml"

print(f"XML dosyası: {xml_path}")
print(f"Dosya mevcut: {xml_path.exists()}")

# Parser oluştur
parser = MicGeometryParser(str(xml_path))

# Parse et
positions = parser.parse()

print(f"\n✓ {parser.num_mics} mikrofon pozisyonu yüklendi")
print(f"Pozisyon matrisi shape: {positions.shape}")

## 2. Geometri Bilgilerini İncele

In [ ]:
info = parser.get_array_info()

print("=" * 50)
print("MİKROFON DİZİSİ BİLGİLERİ")
print("=" * 50)

for key, value in info.items():
    if key == 'bounding_box':
        print(f"\n{key}:")
        for k, v in value.items():
            print(f"  {k}: {v:.6f} m")
    elif key == 'center':
        print(f"\n{key}: [{value[0]:.6f}, {value[1]:.6f}, {value[2]:.6f}] m")
    elif isinstance(value, float):
        print(f"{key}: {value:.6f} m")
    else:
        print(f"{key}: {value}")

print("\n" + "=" * 50)

## 3. Pozisyonları Görselleştir

In [ ]:
%matplotlib inline

# 3D + 2D görselleştirme
parser.visualize()

## 4. Detaylı Pozisyon Tablosu

In [ ]:
print("Mikrofon Pozisyonları (metre)")
print("-" * 60)
print(f"{'Mic ID':<8} {'X':<15} {'Y':<15} {'Z':<15}")
print("-" * 60)

for i in range(positions.shape[1]):
    x, y, z = positions[:, i]
    print(f"{i:<8} {x:<15.6f} {y:<15.6f} {z:<15.6f}")

print("-" * 60)

## 5. Acoular Entegrasyonu Test

In [ ]:
try:
    from acoular import MicGeom
    
    # Acoular objesi oluştur
    mic_geom = parser.to_acoular()
    
    print("✓ Acoular MicGeom objesi başarıyla oluşturuldu")
    print(f"  - Mikrofon sayısı: {mic_geom.num_mics}")
    print(f"  - Pozisyon matrisi shape: {mic_geom.mpos_tot.shape}")
    print(f"  - İlk 3 mikrofon pozisyonu:")
    print(mic_geom.mpos_tot[:, :3])
    
except ImportError:
    print("⚠ Acoular kurulu değil. Lütfen 'pip install acoular' ile kurun.")

## 6. Mikrofonlar Arası Mesafe Analizi

In [ ]:
from scipy.spatial.distance import pdist, squareform

# Tüm mikrofonlar arası mesafeleri hesapla
distances = squareform(pdist(positions.T))

# Köşegen hariç (0'lar)
mask = ~np.eye(distances.shape[0], dtype=bool)
distances_non_zero = distances[mask]

print("Mikrofonlar Arası Mesafe İstatistikleri:")
print("-" * 50)
print(f"Minimum mesafe: {np.min(distances_non_zero):.6f} m")
print(f"Maksimum mesafe: {np.max(distances_non_zero):.6f} m")
print(f"Ortalama mesafe: {np.mean(distances_non_zero):.6f} m")
print(f"Std sapma: {np.std(distances_non_zero):.6f} m")

# Histogram
plt.figure(figsize=(10, 5))
plt.hist(distances_non_zero, bins=30, edgecolor='black', alpha=0.7)
plt.xlabel('Mesafe (m)')
plt.ylabel('Frekans')
plt.title('Mikrofonlar Arası Mesafe Dağılımı')
plt.grid(True, alpha=0.3)
plt.show()

## ✅ Sonuç

Bu notebook başarıyla çalıştıysa:
- ✓ XML dosyası doğru parse ediliyor
- ✓ Mikrofon geometrisi görselleştiriliyor
- ✓ Acoular entegrasyonu çalışıyor

**Sonraki adım:** Audio stream testine geçebiliriz!